In [1]:
import ee
ee.Initialize()
from iteru import *
from ipywidgets import *
from ipyleaflet import GeoJSON
import json

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [3]:
with open ('./examples/aoi.geojson') as file:
    studyarea = json.load(file)
aoi = ee.FeatureCollection(studyarea)
Map.add_ee_layer(aoi,{'color':'blue'},'Area of Interest')

In [79]:
collection= ee.ImageCollection('COPERNICUS/S2_SR')\
.filterBounds(aoi)\
.filterDate('2021-01-01','2021-02-01')

In [80]:
def collection_vis_params(collection):
    min = float(collection.getInfo()['properties']['visualization_0_min'])
    max = float(collection.getInfo()['properties']['visualization_0_max'])
    bands = collection.getInfo()['properties']['visualization_0_bands'].split(',')
    return {'min':min, 'max':max,'bands':bands}

In [134]:
def best_image(year):
    start_date = ee.Date.fromYMD(year,1,1)